In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

!pip install aif360
!pip install BlackBoxAuditing

from aif360.algorithms.preprocessing import DisparateImpactRemover
from aif360.datasets import BinaryLabelDataset

In [134]:
database = pd.read_csv('/content/drive/MyDrive/Pesquisa/Código/LoanData_Bondora.csv')

/tmp/ipython-input-3058922216.py:1: DtypeWarning: Columns (34,38,80,82,83,89,90) have mixed types. Specify dtype option on import or set low_memory=False.
  database = pd.read_csv('/content/drive/MyDrive/Pesquisa/Código/LoanData_Bondora.csv')


In [135]:
database

,ReportAsOfEOD,LoanId,LoanNumber,ListedOnUTC,BiddingStartedOn,BidsPortfolioManager,BidsApi,BidsManual,UserName,NewCreditCustomer,...,PreviousEarlyRepaymentsCountBeforeLoan,GracePeriodStart,GracePeriodEnd,NextPaymentDate,NextPaymentNr,NrOfScheduledPayments,ReScheduledOn,PrincipalDebtServicingCost,InterestAndPenaltyDebtServicingCost,ActiveLateLastPaymentCategory
0,2021-07-20,66AE108B-532B-4BB3-BAB7-0019A46412C1,483449,2016-03-23 16:07:19,2016-03-23 16:07:19,970,1150,5.0,BO965519,False,...,0.0,2019-10-28,2020-01-27,NaN,NaN,NaN,NaN,0.00,51.73,31-60
1,2021-07-20,D152382E-A50D-46ED-8FF2-0053E0C86A70,378148,2015-06-25 11:02:28,2015-06-25 11:02:28,1295,0,1705.0,BOA9K172A,False,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,47.08,180+
2,2021-07-20,87342E13-66CB-483F-833A-007953E50C78,451831,2016-01-14 10:00:21,2016-01-14 10:00:21,2700,565,5835.0,BO7971663,True,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,259.67,1659.60,180+
3,2021-07-20,87227056-6BF9-410C-98D1-008F788E122A,349381,2015-03-24 15:55:44,2015-03-24 15:55:44,1115,0,385.0,BO76151K3,True,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,215.05,180+
4,2021-07-20,2DDE6336-E466-4624-A337-00A0ED1A1468,443082,2015-12-17 10:12:00,2015-12-17 10:12:00,305,0,785.0,BOK423A63,True,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,180+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179230,2021-07-20,D9E714C3-B9B8-4FD1-B1AD-FF55B3AE8A8F,323749,2015-03-02 07:07:05,2015-03-02 07:07:05,2000,0,0.0,BO6492A4A,True,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,18.38,180+
179231,2021-07-20,09999223-3D69-42E3-BAAA-FF7F312AB236,468598,2016-02-17 10:17:35,2016-02-17 10:17:35,6215,70,1155.0,BO542A57A,False,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN
179232,2021-07-20,370193A5-D19C-4A99-97F7-FF95BEA52731,462572,2016-02-05 09:35:26,2016-02-05 09:35:26,430,0,1055.0,BO65A965,False,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN
179233,2021-07-20,98D9FCCE-0FD7-420E-9F7F-FFA960669E8B,396046,2015-08-17 17:23:05,2015-08-17 17:23:05,3000,0,0.0,BO9577433,True,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN


In [136]:
database.shape

(179235, 112)

In [137]:
database = database[database['Status'] != 'Current'] # excluding samples that have status as 'Current'

In [138]:
database['Status'].value_counts()

,count
Status,
Late,68574
Repaid,52887


In [139]:
remove_database = [

     # features related to date, time, and results from other models
    'ReportAsOfEOD', 'LoanId', 'LoanNumber', 'ListedOnUTC', 'BiddingStartedOn',
    'UserName', 'LoanApplicationStartedDate', 'LoanDate', 'FirstPaymentDate',
    'MaturityDate_Original', 'MaturityDate_Last', 'ApplicationSignedHour',
    'ApplicationSignedWeekday', 'MonthlyPaymentDay',
    'ActiveScheduleFirstPaymentReached', 'LastPaymentOn', 'ExpectedLoss',
    'ExpectedReturn', 'ProbabilityOfDefault', 'RecoveryStage',
    'ModelVersion', 'Rating', 'Restructured', 'CreditScoreEsMicroL', 'DateOfBirth',
    'CurrentDebtDaysPrimary', 'DebtOccuredOn', 'CurrentDebtDaysSecondary',
    'DebtOccuredOnForSecondary', 'DefaultDate', 'EL_V0', 'Rating_V0', 'EL_V1',
    'Rating_V1', 'Rating_V2', 'CreditScoreEsEquifaxRisk',
    'CreditScoreFiAsiakasTietoRiskGrade', 'CreditScoreEeMini', 'GracePeriodStart',
    'GracePeriodEnd', 'NextPaymentDate', 'ReScheduledOn',
    'ActiveLateLastPaymentCategory', 'EmploymentDurationCurrentEmployer',

     # Variáveis com alto risco de vazamento identificadas

    'PlannedInterestTillDate', # Juros previstos a serem recebidos até o momento, com base no cronograma de pagamentos
    'PlannedPrincipalTillDate', # Principal (valor emprestado) previsto a ser recebido até o momento, com base no cronograma de pagamentos
    'PrincipalOverdueBySchedule', # Valor do principal em atraso, de acordo com o cronograma atual
    'WorseLateCategory', # Maior número de dias de atraso no pagamento do principal
    'PrincipalBalance', # Saldo devedor do principal (valor restante a ser pago pelo tomador)
    'InterestAndPenaltyBalance', # Juros e multas ainda não pagos
    'NextPaymentNr', # Número sequencial do próximo pagamento
    'ContractEndDate', # Data de término do contrato de empréstimo
    'PrincipalPaymentsMade', # Total de principal já pago
    'InterestAndPenaltyPaymentsMade', # Total de juros e multas já pagos
    'PrincipalWriteOffs', # Valor do principal baixado como prejuízo
    'InterestAndPenaltyWriteOffs', # Juros e multas baixados como prejuízo


]

In [140]:
database = database.drop(columns=remove_database, errors='ignore')

In [141]:
database.shape

(121461, 56)

In [142]:
nullFeatures = []
for col in database.columns:
    nullValues = database[col].isnull().sum()
    nullPercentage = nullValues/database.shape[0]
    if nullPercentage > 0.2:
        print(f"{col} : {nullPercentage*100:.2f}%")
        nullFeatures.append(col)
print(nullFeatures)

County : 100.00%
City : 100.00%
NrOfDependants : 71.91%
EmploymentPosition : 100.00%
WorkExperience : 71.18%
PlannedPrincipalPostDefault : 41.34%
PlannedInterestPostDefault : 41.34%
EAD1 : 41.34%
EAD2 : 41.34%
PrincipalRecovery : 41.34%
InterestRecovery : 41.34%
StageActiveSince : 25.78%
ActiveLateCategory : 38.26%
PreviousRepaymentsBeforeLoan : 32.57%
PreviousEarlyRepaymentsBefoleLoan : 55.86%
NrOfScheduledPayments : 55.10%
PrincipalDebtServicingCost : 40.87%
InterestAndPenaltyDebtServicingCost : 40.87%
['County', 'City', 'NrOfDependants', 'EmploymentPosition', 'WorkExperience', 'PlannedPrincipalPostDefault', 'PlannedInterestPostDefault', 'EAD1', 'EAD2', 'PrincipalRecovery', 'InterestRecovery', 'StageActiveSince', 'ActiveLateCategory', 'PreviousRepaymentsBeforeLoan', 'PreviousEarlyRepaymentsBefoleLoan', 'NrOfScheduledPayments', 'PrincipalDebtServicingCost', 'InterestAndPenaltyDebtServicingCost']


In [143]:
database = database.drop(columns=nullFeatures)

In [144]:
database.shape

(121461, 38)

In [145]:
numerical_features = database.select_dtypes(include=np.number).columns.tolist()
categorical_features = database.select_dtypes(exclude=np.number).columns.tolist()

print("Numerical Fe.atures:")
print(numerical_features)

print("\nCategotic Features:")
categorical_features

Numerical Fe.atures:
['BidsPortfolioManager', 'BidsApi', 'BidsManual', 'VerificationType', 'LanguageCode', 'Age', 'Gender', 'AppliedAmount', 'Amount', 'Interest', 'LoanDuration', 'MonthlyPayment', 'UseOfLoan', 'Education', 'MaritalStatus', 'EmploymentStatus', 'OccupationArea', 'HomeOwnershipType', 'IncomeFromPrincipalEmployer', 'IncomeFromPension', 'IncomeFromFamilyAllowance', 'IncomeFromSocialWelfare', 'IncomeFromLeavePay', 'IncomeFromChildSupport', 'IncomeOther', 'IncomeTotal', 'ExistingLiabilities', 'LiabilitiesTotal', 'RefinanceLiabilities', 'DebtToIncome', 'FreeCash', 'LossGivenDefault', 'NoOfPreviousLoansBeforeLoan', 'AmountOfPreviousLoansBeforeLoan', 'PreviousEarlyRepaymentsCountBeforeLoan']

Categotic Features:


['NewCreditCustomer', 'Country', 'Status']

In [146]:
# Check for null values in numerical features
for col in numerical_features:
    null_count = database[col].isnull().sum()
    if null_count > 0:
        print(f"Feature '{col}' has {null_count} null values.")


Feature 'VerificationType' has 50 null values.
Feature 'Gender' has 45 null values.
Feature 'MonthlyPayment' has 6660 null values.
Feature 'Education' has 50 null values.
Feature 'MaritalStatus' has 50 null values.
Feature 'EmploymentStatus' has 202 null values.
Feature 'OccupationArea' has 91 null values.
Feature 'HomeOwnershipType' has 1657 null values.
Feature 'DebtToIncome' has 50 null values.
Feature 'FreeCash' has 50 null values.
Feature 'LossGivenDefault' has 2638 null values.
Feature 'NoOfPreviousLoansBeforeLoan' has 8 null values.
Feature 'AmountOfPreviousLoansBeforeLoan' has 8 null values.
Feature 'PreviousEarlyRepaymentsCountBeforeLoan' has 8 null values.


In [147]:
# Calculate the median for each numerical feature and fill missing values
for col in numerical_features:
    median_val = database[col].median()
    database[col] = database[col].fillna(median_val)


In [148]:
# Check for null values in categorics features
for col in categorical_features:
    null_count = database[col].isnull().sum()
    if null_count > 0:
        print(f"Feature '{col}' has {null_count} null values.")


In [149]:
database = database[numerical_features + categorical_features]

In [150]:
database['Status'].value_counts()

,count
Status,
Late,68574
Repaid,52887


In [151]:
database['Status'] = database['Status'].map({'Repaid': 0, 'Late': 1}) # 0 -> pago; 1-> não pago

In [152]:
database["Status"].value_counts()

,count
Status,
1,68574
0,52887


In [153]:
from sklearn.preprocessing import LabelEncoder

for col in database.columns:
    if database[col].dtype == 'object':
        database[col] = database[col].astype(str)
        le = LabelEncoder()
        database[col] = le.fit_transform(database[col])

print(database.dtypes)
print(database.head())


BidsPortfolioManager                        int64
BidsApi                                     int64
BidsManual                                float64
VerificationType                          float64
LanguageCode                                int64
Age                                         int64
Gender                                    float64
AppliedAmount                             float64
Amount                                    float64
Interest                                  float64
LoanDuration                                int64
MonthlyPayment                            float64
UseOfLoan                                   int64
Education                                 float64
MaritalStatus                             float64
EmploymentStatus                          float64
OccupationArea                            float64
HomeOwnershipType                         float64
IncomeFromPrincipalEmployer               float64
IncomeFromPension                         float64


In [154]:
database['MaritalStatus']

,MaritalStatus
0,2.0
1,2.0
2,4.0
3,1.0
4,4.0
...,...
179230,1.0
179231,3.0
179232,3.0
179233,1.0


In [155]:
database['Education']

,Education
0,4.0
1,5.0
2,4.0
3,2.0
4,4.0
...,...
179230,3.0
179231,4.0
179232,3.0
179233,3.0


# Aplicação DIR

In [156]:
# Encontrando o grupo menos favorecido de Gender

#masculino = 0; feminino = 1; indefinido = 2

def calcular_di_Gender(database, grupo1, grupo2):
    rate1 = database[database['Gender'] == grupo1]['Status'].mean()
    rate2 = database[database['Gender'] == grupo2]['Status'].mean()
    di = rate1 / rate2
    return di

# Pares que você quer calcular
pares = [(0, 1), (1, 2), (2, 0)]

for g1, g2 in pares:
    di = calcular_di_Gender(database, g1, g2)
    print(f"DI ({g1} / {g2}): {di:.4f}")

DI (0 / 1): 1.1047
DI (1 / 2): 0.6788
DI (2 / 0): 1.3335


In [157]:
# Encontrando o grupo menos favorecido de MaritalStatus

#1 Casado; 2 União estável; 3 Solteiro; 4 Divorciado; 5 Viúvo

def calcular_di_MaritalStatus(database, grupo1, grupo2):
    rate1 = database[database['MaritalStatus'] == grupo1]['Status'].mean()
    rate2 = database[database['MaritalStatus'] == grupo2]['Status'].mean()
    di = rate1 / rate2
    return di

# Pares para calcular
pares = [(1, 2), (1, 3), (1, 4), (1, 5),
 (2, 3), (2, 4), (2, 5),
 (3, 4), (3, 5),
 (4, 5)]

for g1, g2 in pares:
    di = calcular_di_MaritalStatus(database, g1, g2)
    print(f"DI ({g1} / {g2}): {di:.4f}")

DI (1 / 2): 1.2471
DI (1 / 3): 0.9755
DI (1 / 4): 0.9090
DI (1 / 5): 1.0235
DI (2 / 3): 0.7822
DI (2 / 4): 0.7289
DI (2 / 5): 0.8207
DI (3 / 4): 0.9319
DI (3 / 5): 1.0493
DI (4 / 5): 1.1260


In [158]:
# Encontrando o grupo menos favorecido de Education

# 1 Ensino primário; 2 Ensino básico; 3 Educação profissional; 4 Ensino secundário; 5 Ensino superior

def calcular_di_Education(database, grupo1, grupo2):
    rate1 = database[database['Education'] == grupo1]['Status'].mean()
    rate2 = database[database['Education'] == grupo2]['Status'].mean()
    di = rate1 / rate2
    return di

# Pares para calcular
pares = [(1, 2), (1, 3), (1, 4), (1, 5),
 (2, 3), (2, 4), (2, 5),
 (3, 4), (3, 5),
 (4, 5)]

for g1, g2 in pares:
    di = calcular_di_Education(database, g1, g2)
    print(f"DI ({g1} / {g2}): {di:.4f}")



DI (1 / 2): 1.3817
DI (1 / 3): 1.1233
DI (1 / 4): 1.3554
DI (1 / 5): 1.3838
DI (2 / 3): 0.8130
DI (2 / 4): 0.9810
DI (2 / 5): 1.0015
DI (3 / 4): 1.2067
DI (3 / 5): 1.2320
DI (4 / 5): 1.0210


In [159]:
# criando features para aplicação do DIR

# Gender: grupo 2 é desprivilegiado =  1
#database['GenderBinary'] = database['Gender'].apply(lambda x: 1 if x == 2 else 0)

# Age: < 40 é menos favorecido = 1
#database['PrivilegedAge'] = database['Age'].apply(lambda x: 1 if x < 40 else 0)

# MaritalStatus: grupo 2 é desprivilegiado = 1
database['MaritalStatusBinary'] = database['MaritalStatus'].apply(lambda x: 1 if x == 2 else 0)

# Education: grupos 2, 4, 5 são desprivilegiados = 1
#database['EducationBinary'] = database['Education'].apply(lambda x: 1 if x in [2, 4, 5] else 0)


In [160]:
#print(database['PrivilegedAge'].value_counts())


In [161]:
dataset = BinaryLabelDataset(
    df=database,
    label_names=['Status'],
    protected_attribute_names=['MaritalStatusBinary']
)

#protected_attribute_names=['GenderBinary', 'PrivilegedAge', 'MaritalStatusBinary', 'EducationBinary']

dir = DisparateImpactRemover(repair_level=1.0)

database = dir.fit_transform(dataset)

database = database.convert_to_dataframe()[0]


In [162]:
database = database.drop(columns=['MaritalStatus'])

#database = database.drop(columns=['Gender', 'Age', 'MaritalStatus', 'Education'])

## Aplicando one-hot encoding

In [163]:
cols_cat = [
    "VerificationType",
    "LanguageCode",
    "UseOfLoan",
    "EmploymentStatus",
    "OccupationArea",
    "HomeOwnershipType",
    "NewCreditCustomer",
    "Country"
]

# Aplica one-hot encoding
database = pd.get_dummies(
    database,
    columns=cols_cat,
    prefix=cols_cat,
    drop_first=True  # remove a primeira categoria de cada variável para evitar multicolinearidade
)

for col in database.columns:
    if database[col].dtype == 'bool':
        database[col] = database[col].astype(int)


In [164]:
%matplotlib inline

corr_matrix = database.corr().abs()

# Pega apenas a parte superior da matriz (sem diagonais duplicadas)
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Identifica colunas com correlação >= 0.7 e registra com quem elas estão mais correlacionadas
colunas_remover = []
print("Colunas com alta correlação (>= 0.7) e com quem estão mais correlacionadas:\n")
for col in upper.columns:
    correlacoes_altas = upper[col][upper[col] >= 0.7]
    if not correlacoes_altas.empty:
        mais_correlacionada = correlacoes_altas.idxmax()
        valor = correlacoes_altas.max()
        print(f"- {col} está altamente correlacionada com {mais_correlacionada} (correlação = {valor:.2f})")
        colunas_remover.append(col)

# Remove as colunas altamente correlacionadas
database.drop(columns=colunas_remover, inplace=True)

# Plot da nova matriz de correlação
plt.figure(figsize=(70, 70))
sns.heatmap(database.corr(), annot=True, cmap='viridis')
plt.title("Matriz de Correlação (sem colunas altamente correlacionadas)")
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [165]:
database.to_csv('/content/drive/MyDrive/Pesquisa/Código/Aplicação/DIR/preprocessed_databaseDIR.csv', index=False)